<a href="https://colab.research.google.com/github/ThistleAna/snowflakes/blob/main/Car_Sales_RNNModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Recurrent Neural Network Models

In [2]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))
# make a prediction
row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))

(91, 5, 1) (12, 5, 1) (91,) (12,)
Epoch 1/350
3/3 - 2s - loss: 166373952.0000 - mae: 11326.9160 - val_loss: 108694104.0000 - val_mae: 8704.3115
Epoch 2/350
3/3 - 0s - loss: 50155904.0000 - mae: 5721.9194 - val_loss: 22828726.0000 - val_mae: 3959.0286
Epoch 3/350
3/3 - 0s - loss: 31698410.0000 - mae: 4606.4438 - val_loss: 28333682.0000 - val_mae: 4675.4888
Epoch 4/350
3/3 - 0s - loss: 27504572.0000 - mae: 4032.8513 - val_loss: 16035743.0000 - val_mae: 2672.7239
Epoch 5/350
3/3 - 0s - loss: 29320388.0000 - mae: 4143.1011 - val_loss: 38967328.0000 - val_mae: 4046.2004
Epoch 6/350
3/3 - 0s - loss: 24648436.0000 - mae: 3755.7788 - val_loss: 32085440.0000 - val_mae: 3835.5430
Epoch 7/350
3/3 - 0s - loss: 18616268.0000 - mae: 3335.2019 - val_loss: 26093286.0000 - val_mae: 3628.4463
Epoch 8/350
3/3 - 0s - loss: 17898422.0000 - mae: 3349.0293 - val_loss: 19223706.0000 - val_mae: 3459.3674
Epoch 9/350
3/3 - 0s - loss: 15466708.0000 - mae: 3169.3269 - val_loss: 18532400.0000 - val_mae: 3324.3145


In [ ]:
# result from the first run MSE: 10895209.000, RMSE: 3300.789, MAE: 2576.508
# Predicted: 17387.074

# result from the second run MSE:12357601.000, RMSE:3515.338, MAE: 2858.094
# Predicted: 16882.143
# https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/